In [ ]:
!pip install ultralytics
!pip install deep_sort_realtime
!pip install opencv-python


In [ ]:
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
#best one
model = YOLO("best (1).pt")
model.to("cuda")

from ultralytics import YOLO

yaml_content = """tracker_type: botsort
track_high_thresh: 0.6
track_low_thresh: 0.1
match_thresh: 0.8
track_buffer: 30
gmc: False
cmc_method: 'greedy'
ablation: False
mot20: False
"""
with open("botsort.yaml", "w") as f:
    f.write(yaml_content)


model = YOLO("best (1).pt")
model.to("cuda")

results = model.track(
    source="0711(1).mp4",
    tracker="botsort.yaml",
    stream=True,
    conf=0.5,
    iou=0.7,
    persist=True,
    save=True
)

video_path = "15sec.mp4"
cap = cv2.VideoCapture(video_path)


out = cv2.VideoWriter("tracked_output.mp4",
                      cv2.VideoWriter_fourcc(*"mp4v"),
                      cap.get(cv2.CAP_PROP_FPS),
                      (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
                       int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = yolo_model(frame, verbose=False)[0]

    detections = []
    for box in results.boxes:
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
        conf = float(box.conf[0].cpu().numpy())
        cls = int(box.cls[0].cpu().numpy())
        detections.append(([x1, y1, x2 - x1, y2 - y1], conf, cls))

    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        ltrb = track.to_ltrb()
        x1, y1, x2, y2 = map(int, ltrb)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f'ID: {track_id}', (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

    out.write(frame)

cap.release()
out.release()


In [ ]:
from ultralytics import YOLO
model = YOLO("best (1).pt")
model.to("cuda")

results = model.track(
    source="0711(1).mp4",
    tracker="botsort.yaml",
    stream=False,
    conf=0.5,
    iou=0.7,
    persist=True,
    save=True,
    name="tracked_output"
)



WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/4500) /content/0711(1).mp4: 384x640 16 players, 1 referee, 76.6ms
video 1/1 (frame 2/4500) /content/0711(1).mp4: 384x640 16 players, 1 referee, 51.8ms
video 1/1 (frame 3/4500) /content/0711(1).mp4: 384x640 16 players, 1 referee, 51.8ms
video 1/1 (frame 4/4500) /content/0711(1).mp4: 384x640 16 players, 1 referee, 51.8ms
video 1/1 (frame 5/4500) /content/0711(1).mp4: 384x640 16 players, 1 referee, 51.2ms
video 1/1 (frame 6/4500) /conte

In [ ]:
#please note that in google colab complete cant be done due to gpu overhead
#So the rest of training is done on kaggle notebooks